## Import

In [202]:
import re
from pathlib import Path

In [203]:
import pandas as pd
import altair as alt

In [204]:
import altair_theme

In [205]:
alt.themes.register('palewire', altair_theme.theme)
alt.themes.enable('palewire')

ThemeRegistry.enable('palewire')

## Extract

Convert the PDF document we received via FOIA into a text file

In [206]:
!pdftotext ./input/RemotePilotListSep2022.pdf input/list.txt

Take a peek at it

In [207]:
!head input/list.txt

LastName FMName DOI
AADLAND LAUREN EL 2/23/2018
AAKER
ALEXANDRA 2/14/2017
AAKRE
KEITH ELME 7/31/2020
AAL‐ANUBI YHOSHUA L
3/9/2021
AALBERS MICHAEL T 5/13/2021
AAMER


Get it into Python

In [208]:
input_dir = Path("./input")

In [209]:
with open(input_dir / "list.txt", "r") as f:
    txt_data = f.read().split()

In [210]:
txt_data[:10]

['LastName',
 'FMName',
 'DOI',
 'AADLAND',
 'LAUREN',
 'EL',
 '2/23/2018',
 'AAKER',
 'ALEXANDRA',
 '2/14/2017']

Cut the header row

In [211]:
txt_data.pop(0)
txt_data.pop(0)
txt_data.pop(0)

'DOI'

In [212]:
txt_data[:10]

['AADLAND',
 'LAUREN',
 'EL',
 '2/23/2018',
 'AAKER',
 'ALEXANDRA',
 '2/14/2017',
 'AAKRE',
 'KEITH',
 'ELME']

## Consolidate

Group rows

In [213]:
date_pattern = re.compile("\d{1,2}/\d{1,2}/\d{4}")

In [216]:
def parse_tokens(token_list):
    row_list = []
    row = dict()
    name = ""
    for t in token_list:
        if date_pattern.search(t):
            row['name'] = (name.strip() + " " + date_pattern.sub("", t)).strip()
            row['date'] = date_pattern.search(t).group(0)
            row_list.append(row)
            row = dict()
            name = ""
        else:
            name += " " + t
    return row_list

Consolidate them into a single dataframe

In [217]:
df = pd.DataFrame(data_list)

In [218]:
df.head()

,name,date
0,AADLAND LAUREN EL,2/23/2018
1,AAKER ALEXANDRA,2/14/2017
2,AAKRE KEITH ELME,7/31/2020
3,AAL‐ANUBI YHOSHUA L,3/9/2021
4,AALBERS MICHAEL T,5/13/2021


In [219]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 292562 entries, 0 to 292561
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   name    292562 non-null  object
 1   date    292562 non-null  object
dtypes: object(2)
memory usage: 4.5+ MB


Parse dates

In [220]:
df['date'] = pd.to_datetime(df['date'])

Pull out a couple date parts

In [221]:
df['year'] = df['date'].dt.year

In [222]:
df['yearmonth'] = pd.to_datetime(df['date'].dt.strftime("%Y-%m") + "-01")

## Aggregate

Annual totals

In [223]:
by_year = df.groupby("year").size().rename("n").reset_index().sort_values("year")

In [224]:
by_year

,year,n
0,2016,22193
1,2017,44907
2,2018,42812
3,2019,46623
4,2020,45400
5,2021,54722
6,2022,35905


Monthly totals

In [225]:
by_yearmonth = df.groupby("yearmonth").size().rename("n").reset_index().sort_values("yearmonth")

With a trend line

In [226]:
by_yearmonth['3_month_rolling_average'] = by_yearmonth.n.rolling(3).mean()

And a year over year comparison

In [227]:
by_yearmonth['yoy_change'] = by_yearmonth.n.diff(12)

In [228]:
by_yearmonth.head()

,yearmonth,n,3_month_rolling_average,yoy_change
0,2016-08-01,2595,NaN,NaN
1,2016-09-01,8710,NaN,NaN
2,2016-10-01,4432,5245.666667,NaN
3,2016-11-01,3294,5478.666667,NaN
4,2016-12-01,3162,3629.333333,NaN


In [229]:
by_yearmonth.tail()

,yearmonth,n,3_month_rolling_average,yoy_change
69,2022-05-01,5414,5249.000000,386.0
70,2022-06-01,5010,5099.666667,-3.0
71,2022-07-01,4699,5041.000000,128.0
72,2022-08-01,1456,3721.666667,-3338.0
73,2022-09-01,311,2155.333333,-3768.0


Clip extremes on each end of the time range

In [230]:
trimmed_yearmonth = by_yearmonth[
    (by_yearmonth['yearmonth'] > "2016-12-01") &
    (by_yearmonth['yearmonth'] < "2022-08-01")
]

In [231]:
trimmed_yearmonth.head()

,yearmonth,n,3_month_rolling_average,yoy_change
5,2017-01-01,3988,3481.333333,NaN
6,2017-02-01,3990,3713.333333,NaN
7,2017-03-01,4940,4306.000000,NaN
8,2017-04-01,4070,4333.333333,NaN
9,2017-05-01,4117,4375.666667,NaN


In [232]:
trimmed_yearmonth.tail()

,yearmonth,n,3_month_rolling_average,yoy_change
67,2022-03-01,5458,4713.333333,-206.0
68,2022-04-01,4875,4927.000000,-384.0
69,2022-05-01,5414,5249.000000,386.0
70,2022-06-01,5010,5099.666667,-3.0
71,2022-07-01,4699,5041.000000,128.0


## Chart

In [233]:
chart = alt.Chart(trimmed_yearmonth).encode(
    x=alt.X(
        "yearmonth:T",
        title=None,
        timeUnit="yearmonth",
    ),
)
    
bars = chart.mark_bar(opacity=0.8).encode(
    y=alt.Y("n:Q", title=None),
    tooltip=["yearmonth", "n"]
)

line = chart.mark_line(color=altair_theme.palette['ramp'][3]).encode(
    y=alt.Y("3_month_rolling_average:Q")
)

(bars + line).properties(
    title="New FAA drone licenses by month",
    width=500
).configure_axisX(grid=False)

/home/palewire/.local/share/virtualenvs/faa-drone-license-analysis-2lpWnKuy/lib/python3.9/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.LayerChart(...)

In [234]:
chart = alt.Chart(trimmed_yearmonth[trimmed_yearmonth.yearmonth.dt.year > 2018]).encode(
    x=alt.X("yearmonth:T", title=None, timeUnit="yearmonth"),
)
    
bars = chart.mark_bar(color="lightblue").encode(
    y=alt.Y("yoy_change:Q", title=None),
    color=alt.condition(
        alt.datum.yoy_change > 0,
        alt.value(altair_theme.palette['ramp'][4]),  # The positive color
        alt.value(altair_theme.palette['accent'])  # The negative color
    ),
    tooltip=["yearmonth", "n", "yoy_change"]
)

(bars).properties(
    title="New FAA drone licenses year over year",
    width=500
).configure_axisX(grid=False)

alt.Chart(...)

## Export

In [235]:
df.to_csv("output/licenses.csv", index=False)

In [236]:
by_yearmonth.to_csv("output/monthly-totals.csv", index=False)